## 一、数据集切分
### 交叉验证
10W训练集分成8W和2W,8W用于第一层模型交叉验证，2W用于第二层模型交叉验证

## 二、算法流程
### 预处理
1. 用LR填充空缺目标值，合并train test数据集
2. 训练dbow和dm ２种doc2vec特征

### 训练第一层模型
1. 训练tfidf-lr模型
2. 训练dbow-nn模型
3. 训练dm-nn模型

### 训练第二层模型
1. 训练 xgb-ens模型

In [2]:
'''1.concat train data and test data 
   2.use lr to fill null label'''

import pandas as pd
import numpy as np
import jieba
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold,cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
import pickle
import cfg

#----------------------load data--------------------------------
df_tr = []
for i,line in enumerate(open(cfg.data_path + 'user_tag_query.10W.TRAIN',encoding='GB18030')):
    segs = line.split('\t')
    row = {}
    row['Id'] = segs[0]
    row['age'] = int(segs[1])
    row['gender'] = int(segs[2])
    row['Education'] = int(segs[3])
    row['query'] = '\t'.join(segs[4:])
    df_tr.append(row)
df_tr = pd.DataFrame(df_tr)

df_te=[]
for i,line in enumerate(open(cfg.data_path + 'user_tag_query.10W.TEST',encoding='GB18030')):
    segs = line.split('\t')
    row = {}
    row['Id'] = segs[0]
    row['query'] = '\t'.join(segs[1:])
    df_te.append(row)
df_te = pd.DataFrame(df_te)

print(df_tr.shape)
print(df_te.shape)

df_all = df_tr
# df_all = pd.concat([df_tr,df_te]).fillna(1)
# df_all.index = range(len(df_all))

for lb in ['Education','age','gender']:
    df_all[lb] = df_all[lb] - 1
    print(df_all.iloc[:100000][lb].value_counts())
    
class Tokenizer():
    def __init__(self):
        self.n = 0
    def __call__(self,line):
        tokens = []
        for query in line.split('\t'):
            words = [word for word in jieba.cut(query)]
            for gram in [1,2]:
                for i in range(len(words) - gram + 1):
                    tokens += ["_*_".join(words[i:i+gram])]
        if np.random.rand() < 0.00001:
            print(line)
            print('='*20)
            print(tokens)
        self.n += 1
        if self.n%10000==0:
            print(self.n,end=' ')
        return tokens    

tfv = TfidfVectorizer(tokenizer=Tokenizer(),min_df=3,max_df=0.95,sublinear_tf=True)
X_sp = tfv.fit_transform(df_all['query'])
# pickle.dump(X_sp,open(root + 'tfidf_10W.pkl','wb'))
print(len(tfv.vocabulary_))
X_all = X_sp

#-----------------------------fill nan-------------------------------------
'''填充空值'''
for lb,idx in [('Education',0),('age',2),('gender',3)]:
    tr = np.where(df_all[lb]!=-1)[0]
    va = np.where(df_all[lb]==-1)[0]
lb = 'Education'
idx = 0
tr = np.where(df_all[lb]!=-1)[0]
va = np.where(df_all[lb]==-1)[0]
df_all.iloc[va,idx] = LogisticRegression(C=1).fit(X_all[tr],df_all.iloc[tr,idx]).predict(X_all[va])

lb = 'age'
idx = 2
tr = np.where(df_all[lb]!=-1)[0]
va = np.where(df_all[lb]==-1)[0]
df_all.iloc[va,idx] = LogisticRegression(C=2).fit(X_all[tr],df_all.iloc[tr,idx]).predict(X_all[va])

lb = 'gender'
idx = 3
tr = np.where(df_all[lb]!=-1)[0]
va = np.where(df_all[lb]==-1)[0]
df_all.iloc[va,idx] = LogisticRegression(C=2).fit(X_all[tr],df_all.iloc[tr,idx]).predict(X_all[va])

df_all = pd.concat([df_all,df_te]).fillna(0)
df_all.to_csv(cfg.data_path + 'all_v2.csv',index=None,encoding='utf8')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\sun\AppData\Local\Temp\jieba.cache


(100000, 5)
(100000, 2)
 4    37107
 3    28148
 2    18858
-1     9280
 5     5693
 1      560
 0      354
Name: Education, dtype: int64
 0    38996
 1    26744
 2    18529
 3    10654
 4     2922
-1     1666
 5      489
Name: age, dtype: int64
 0    56976
 1    40869
-1     2155
Name: gender, dtype: int64


Loading model cost 0.660 seconds.
Prefix dict has been built succesfully.


10000 20000 30000 频来入梦的频是什么意思	涨的多音字	王烁阳	寒暄的意思	陈昊	徐娇	望而生畏的畏是什么意思	狂风怒号的号是什么意思	校草大人求放过	李爱叶	千什么万仞	竣工的近义词	杨慧	读后感	饶组词	2016qq网名最新版女	作文那是一次勇敢的尝试开头	凌潇肃	女生日记读后感400字	勉怎么组词	qq个性说说大全	昵称女生简短好听2字	读后感窗边的小豆豆400字	烟台橡树湾房子敢买吗	城南旧事读后感400字	女生日记读后感400字,200字写感受	啼啭的意思	勉怎么读	我悠悠忽忽地漫游了一个下午	固安自行车主题公园	我不是潘金莲改档	世纪星手机和誉品手机哪个更好	描写昆虫的段落	清组词	窗边的小豆豆	tfboys动漫头像	频的意思	陈昊海绵宝宝	跻身是什么意思	烟台橡树湾小区	恶魔少爷别吻我	涨的多音字组词和拼音	返璞归真是什么意思	凌潇肃王志	变幻莫测的近义词	韵组词	透明的近义词	怅望灰天的怅望是什么意思	描写萤火虫的段落	描写昆虫的作文200字	寒暄的暄是什么意思	望而生畏是什么意思	胡先煦	月球模型被吹走	奇迹的意思	书信格式	雨中的颜色	衡量是什么意思	qq女生网名	描写昆虫的段落100字	观察蝴蝶的作文(用上拟人的修辞手法)150字	马宇佟	关于雨的音乐	观察蜻蜓的作文(用上拟人的修辞手法)150字	频的意思是什么	憧憬是什么意思	第一次尝试作文400	读女生日记读后感	固安第一小学六一图片	寝不安席的安是什么意思	余韵的意思	长方形的面积公式	草虫的村落中的村落指什么	王智凌潇潇	绵亘蜿蜒是什么意思	耀眼的近义词	王晶晶	世纪星手机图片	2.4米超级月饼	海绵宝宝的配音是谁	静谧的近义词	固安有哪些学校	2016qq网名	王智	尖端是什么意思	陈浩	凌潇肃姚晨离婚原因	固安第一小学现在图片	tfboys动漫萌头像	王智性感照	2016qq网名女生	勤组词	固安第一小学图片	读女生日记有感400字	读后感窗边的小豆豆	2016qq女生网名	阻挠的近义词	涨的多音字组词大全	章子怡催汪峰写歌	昵称女生简短好听	固安第一小学	作文那是一次勇敢的尝试	烟台	温博研	赵今麦	草丛的村落	草丛的村落中的村落指什么	王智老公	织上天的织什么意思	要挟的近义词	社会主义核心价值观手抄报	宋玉枝	固安公园	我悠悠忽忽地漫游了一个下午中的悠悠忽忽是

In [15]:
df_tr['gender'].value_counts()

0    58311
1    41689
Name: gender, dtype: int64

In [12]:
df_all['gender'].value_counts()

0.0    158311
1.0     41689
Name: gender, dtype: int64

In [4]:
df_tr.head(5)

,Education,Id,age,gender,query
0,3,22DD920316420BE2DF8D6EE651BA174B,0,0,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,2,43CC3AF5A8D6430A3B572337A889AFE4,1,0,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,4,E97654BFF5570E2CCD433EA6128EAC19,3,0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,2,6931EFC26D229CCFCEA125D3F3C21E57,3,1,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,3,E780470C3BB0D340334BD08CDCC3C71A,1,1,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


In [5]:
df_te.head(5)

,Id,query
0,ED89D43B9F602F96D96C25255F7C228C,陈学冬将出的作品\t刘昊然与谭松韵\t211学校的分数线\t谁唱的味道好听\t吻戏是真吻还是...
1,83C3B7B4AAF8074655A8079F561A76D6,e的0.0052次方\tqq怎么快速提现\t绝色倾城飞烟\t马克思主义基本原理概论\t康世恩...
2,CA9F675A024FB2353849350A35CF8B0F,黑暗文\tlpl夏季赛\t大富豪电玩城\t英雄联盟之电竞称王\t手机怎么扫描手机上的二维码\...
3,DE45B5C4E57AAEBCF3FDFA2A774093BF,中秋水库钓鱼\t鱼竿\t用蚯蚓钓鱼怎样调漂\t传统钓\t3号鱼钩\t鲫鱼汤的做法大全\t鱼饵...
4,406A681FB3DF81EC0E561796AE50AE50,号码吉凶\t退休干部死后配偶\t郫县有哪些大学\t胜利油田属于中石化还是中石油\t苏珊米勒狮...


In [3]:
df_all.head(5)

,Education,Id,age,gender,query
0,3.0,22DD920316420BE2DF8D6EE651BA174B,0.0,0.0,柔和双沟\t女生\t中财网首页 财经\thttp://pan.baidu.com/s/1pl...
1,2.0,43CC3AF5A8D6430A3B572337A889AFE4,1.0,0.0,"广州厨宝烤箱\t世情薄,人情恶,雨送黄昏花易落,晓风干,泪痕\t厦门酒店用品批发市场\t我只..."
2,4.0,E97654BFF5570E2CCD433EA6128EAC19,3.0,0.0,钻石之泪耳机\t盘锦到沈阳\t旅顺公交\t辽宁阜新车牌\tbaidu\tk715\tk716...
3,2.0,6931EFC26D229CCFCEA125D3F3C21E57,3.0,1.0,最受欢迎狗狗排行榜\t舶怎么读\t场景描 写范例\t三维绘图软件\t枣和酸奶能一起吃吗\t好...
4,3.0,E780470C3BB0D340334BD08CDCC3C71A,1.0,1.0,干槽症能自愈吗\t太太万岁叶舒心去没去美国\t干槽症\t右眼皮下面一直跳是怎么回事\t麦当劳...


In [17]:
'''train dbow/dm for education/age/gender'''

import pandas as pd
import jieba
from datetime import datetime
from collections import namedtuple
from gensim.models.doc2vec import Doc2Vec
from collections import OrderedDict
import subprocess
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import codecs
import cfg
import numpy as np

df_all = pd.read_csv(cfg.data_path + 'all_v2.csv',encoding='utf8')
# -------------------add row number to query----------------------
doc_f = codecs.open('alldata-id.txt','w',encoding='utf8')
for i,queries in enumerate(df_all.iloc[:100000]['query']):
    words = []
    for query in queries.split('\t'):
        words.extend(list(jieba.cut(query)))
    tags = [i]
    if i % 10000 == 0:
        print(datetime.now(),i)
    doc_f.write('_*{} {}'.format(i,' '.join(words)))
doc_f.close()

C:\Users\sun\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


2018-09-18 16:58:24.014811 0
2018-09-18 16:59:13.445107 10000
2018-09-18 17:00:02.583493 20000
2018-09-18 17:00:51.289301 30000
2018-09-18 17:01:39.779893 40000
2018-09-18 17:02:28.438270 50000
2018-09-18 17:03:17.842126 60000
2018-09-18 17:04:07.605373 70000
2018-09-18 17:04:56.875287 80000
2018-09-18 17:05:45.370876 90000


In [18]:
SentimentDocument = namedtuple('SentimentDocument', 'words tags')
class Doc_list(object):
    def __init__(self,f):
        self.f = f
    def __iter__(self):
        for i,line in enumerate(codecs.open(self.f,encoding='utf8')):
            words = line.split()
            tags = [int(words[0][2:])]
            words = words[1:]
            yield SentimentDocument(words,tags)

In [19]:
df_lb = pd.read_csv(cfg.data_path + 'all_v2.csv',usecols=['Education','age','gender'],nrows=100000)
ys = {}
for lb in ['Education','age','gender']:
    ys[lb] = np.array(df_lb[lb])

d2v = Doc2Vec(dm=0, size=300, negative=5, hs=0, min_count=3, window=30,sample=1e-5,
              workers=6,alpha=0.025,min_alpha=0.025,epochs=1)
doc_list = Doc_list('alldata-id.txt')
d2v.build_vocab(doc_list)
    
# -------------------train dbow doc2vec---------------------------------------------
for i in range(10):
    print(datetime.now(),'pass:',i + 1)
    doc_list = Doc_list('alldata-id.txt')
    d2v.train(doc_list,total_examples=d2v.corpus_count, epochs=d2v.epochs)
    X_d2v = np.array([d2v.docvecs[i] for i in range(100000)])
    for lb in ["Education",'age','gender']:
        scores = cross_val_score(LogisticRegression(C=3),X_d2v,ys[lb],cv=5)
        print('dbow',lb,scores,np.mean(scores))
d2v.save(cfg.data_path + 'dbow_d2v.model')
print(datetime.now(),'save done')

C:\Users\sun\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


2018-09-18 17:11:12.114198 pass: 1
dbow Education [0.42470753 0.48145185 0.52222389 0.56110611 0.56453468] 0.510804812736049
dbow age [0.39866013 0.43925607 0.47975    0.52072604 0.52817923] 0.4733142943130019
dbow gender [0.58412079 0.66745    0.77885    0.8023     0.7960898 ] 0.7257621196901053
2018-09-18 17:23:31.386536 pass: 2
dbow Education [0.58264174 0.59279072 0.60306985 0.60966097 0.60229034] 0.5980907225820283
dbow age [0.52884712 0.53824618 0.55       0.56537827 0.57088563] 0.5506714384858611
dbow gender [0.81210939 0.8144     0.8237     0.82945    0.82219111] 0.8203701008171503
2018-09-18 17:41:54.435543 pass: 3
dbow Education [0.60978902 0.61478852 0.61516924 0.6229623  0.61709256] 0.6159603287795808
dbow age [0.55974403 0.56444356 0.5701     0.57637882 0.57763665] 0.5696606091359295
dbow gender [0.82500875 0.82375    0.8309     0.8322     0.82889144] 0.8281500388269502
2018-09-18 17:59:43.919977 pass: 4
dbow Education [0.62458754 0.62228777 0.62071896 0.62561256 0.6250437

In [23]:
d2v = Doc2Vec(dm=1, size=300, negative=5, hs=0, min_count=3, window=10,sample=1e-5,
              workers=6,alpha=0.05,min_alpha=0.025,epochs=1)
doc_list = Doc_list('alldata-id.txt')
d2v.build_vocab(doc_list)

# ---------------train dm doc2vec-----------------------------------------------------
for i in range(10):
    print(datetime.now(),'pass:',i)
#     run_cmd('shuf alldata-id.txt > alldata-id-shuf.txt')
    doc_list = Doc_list('alldata-id.txt')
    d2v.train(doc_list,total_examples=d2v.corpus_count, epochs=d2v.epochs)
    X_d2v = np.array([d2v.docvecs[i] for i in range(100000)])
    for lb in ["Education",'age','gender']:
        scores = cross_val_score(LogisticRegression(C=3),X_d2v,ys[lb],cv=5)
        print('dm',lb,scores,np.mean(scores))
d2v.save(cfg.data_path + 'dm_d2v.model')
print(datetime.now(),'save done')

C:\Users\sun\Anaconda3\lib\site-packages\gensim\models\doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


2018-09-18 19:06:51.248945 pass: 0
dm Education [0.44670533 0.53619638 0.56162192 0.55960596 0.54518178] 0.5298622733191444
dm age [0.42425757 0.49760024 0.51505    0.51512576 0.50022503] 0.4904517208522911
dm gender [0.63576821 0.76865    0.78575    0.77905    0.76303815] 0.7464512726994031
2018-09-18 19:23:45.742218 pass: 1
dm Education [0.57269273 0.58844116 0.58662067 0.58550855 0.57028554] 0.5807097298528802
dm age [0.52444756 0.54009599 0.5408     0.53637682 0.52292844] 0.5329297607504534
dm gender [0.77651117 0.7974     0.80025    0.796      0.78458923] 0.7909500807805501
2018-09-18 19:38:53.010901 pass: 2
dm Education [0.58934107 0.59774023 0.59587021 0.59365937 0.58123719] 0.5915696099749838
dm age [0.54334567 0.55084492 0.5497     0.54497725 0.53418013] 0.5446095913646803
dm gender [0.79506025 0.8057     0.8075     0.8038     0.79073954] 0.8005599567928998
2018-09-18 19:48:24.878007 pass: 3
dm Education [0.59999    0.60263974 0.60091995 0.59810981 0.58923839] 0.59817957755351

In [25]:
d2v = Doc2Vec.load(cfg.data_path + 'dm_d2v.model')
for i in range(10):
    print(datetime.now(),'pass:',i)
#     run_cmd('shuf alldata-id.txt > alldata-id-shuf.txt')
    doc_list = Doc_list('alldata-id.txt')
    d2v.train(doc_list,total_examples=d2v.corpus_count, epochs=d2v.epochs)
    X_d2v = np.array([d2v.docvecs[i] for i in range(100000)])
    for lb in ["Education",'age','gender']:
        scores = cross_val_score(LogisticRegression(C=3),X_d2v,ys[lb],cv=5)
        print('dm',lb,scores,np.mean(scores))
d2v.save(cfg.data_path + 'dm_d2v.model')
print(datetime.now(),'save done')

2018-09-18 20:38:37.330051 pass: 0
dm Education [0.61333867 0.61888811 0.61776911 0.6179618  0.61109166] 0.6158098697591743
dm age [0.57194281 0.57454255 0.5704     0.57122856 0.56273441] 0.5701696646108898
dm gender [0.8180591  0.8177     0.82405    0.8241     0.81424071] 0.8196299618161499
2018-09-18 20:44:31.884588 pass: 1
dm Education [0.61393861 0.61968803 0.61921904 0.61951195 0.61259189] 0.6169899032725501
dm age [0.57224278 0.57519248 0.5708     0.57187859 0.5619843 ] 0.5704196296097391
dm gender [0.81770911 0.8176     0.82475    0.8225     0.81514076] 0.8195399743164249
2018-09-18 20:50:26.656293 pass: 2
dm Education [0.61648835 0.62128787 0.61966902 0.6219622  0.61614242] 0.6191099713019522
dm age [0.57359264 0.57684232 0.5721     0.57297865 0.5646347 ] 0.5720296601282154
dm gender [0.81840908 0.8177     0.8252     0.8246     0.81674084] 0.8205299833175749
2018-09-18 20:56:19.028115 pass: 3
dm Education [0.61738826 0.62278772 0.61881906 0.6209621  0.61834275] 0.61965997781422

In [26]:
print(d2v.epochs)

1


In [27]:
'''tfidf-lr stack for education/age/gender'''

import pandas as pd
import numpy as np
import jieba
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import KFold
from datetime import datetime
import cfg

#-----------------------myfunc-----------------------
def myAcc(y_true,y_pred):
    y_pred = np.argmax(y_pred,axis=1)
    return np.mean(y_true == y_pred)
#-----------------------load data--------------------

df_all = pd.read_csv(cfg.data_path + 'all_v2.csv',encoding='utf8',nrows=100000)
ys = {}
for label in ['Education','age','gender']:
    ys[label] = np.array(df_all[label])

class Tokenizer():
    def __init__(self):
        self.n = 0
    def __call__(self,line):
        tokens = []
        for query in line.split('\t'):
            words = [word for word in jieba.cut(query)]
            for gram in [1,2]:
                for i in range(len(words) - gram + 1):
                    tokens += ["_*_".join(words[i:i+gram])]
        if np.random.rand() < 0.00001:
            print(line)
            print('='*20)
            print(tokens)
        self.n += 1
        if self.n%10000==0:
            print(self.n)
        return tokens

tfv = TfidfVectorizer(tokenizer=Tokenizer(),min_df=3,max_df=0.95,sublinear_tf=True)
X_sp = tfv.fit_transform(df_all['query'])
pickle.dump(X_sp,open(cfg.data_path + 'tfidf_10W.feat','wb'))

10000
斯卡哈	我们之中	张宏胜	pandakill第二季	为什么叫pdd扔蛇狂魔	张宏圣	金轮转生爆	二代火影的雷神之剑	真三国无双7猛将传吕布传if结局	为什么叫pdd嫖老师	fate斯卡哈	守候高塔上的公主	真三国无双7猛将传唤醒技能	仙剑vr版	办公室打游戏,我只能说一句话,哥们带我	三国杀徐晃	阿尔托莉雅拔出石中剑	元十三限	直至死亡	战棋游戏,讲封神榜的	战旗桃子和熊猫桃子	最强反派糸统	自知之明	真三国无双7猛将传吕布传三处老虎	九尾双狮掌	皆非歌词	银行开通网银盾流程	南山南歌词	我生君老,锦绣天下你却看不到	炉石传说打玛克扎尔王子用牺牲契约	快乐时光影院	河图三世故事	我的学妹不可能那么萌吧	真三国无双7猛将传樊城之战怎么选徐庶	百度输入法手机版	择天记	坚毅忍传	拳皇14剧情	织女竟给夏侯四十一用歹毒手法制住了,而他?uno牌的玩法	炉石传说双人现开赛战报	你以为我是谁	熊猫tv狼人杀第二季	白色相簿	七里香歌词	东方月初	阵法	炉石传说怎么得到泰兰德	地下城堡2熊地精宝藏	东华理工枫林校区到江西师范大学	择天记电视剧播出时间	真三国无双7猛将传动画怎么获得	熊猫狼人杀第二季	离歌歌词	国足2-3不敌韩国	守候高塔上的公主txt	战网官网	起点中文网	地下城堡2攻略	神话版三国	真三国无双7猛将传动画	何休	无双大蛇z吴第三关逃脱地点	桃子去了战旗	东方剑客奇谭传	重生犬夜叉之修罗丸	来吧冠军	中国建设银行	穿越犬夜叉之我是桔梗	寄生兽电影图片	炉石传说龙牧	im	bigbang	阴天快乐歌词	幻想乡战记	阿求	火影之鸣人风流	公主,如果你打出这张牌你就会出局	dota是暴雪的吗	放置江湖	炉石传说官网积分兑换	大宋提刑官片尾曲	真姬	二尾人柱力	曹宁	炉石传说官网	折木奉太郎	三鞭道人	有整片星空和一只老狗	东华理工大学官网	真三国无双7猛将传吕布传	游戏王5ds漫画和动画的区别	九九八十一歌曲	九九八十一	舞娘歌词	炉石传说怎么获得积分	真三国无双7猛将传吕布传结局	起点	英灵斯卡哈	真三国无双7猛将传护卫支援按键	熊猫狼人杀第二季改名	思乡未敢听琵琶	太极螺旋丸	superliae第二季	苹果qq怎么设置红包提醒功能	三国之我是丁原	无双大蛇z切换地图	超凡兵王	情歌王歌词	陈长生百度百科	炉石传说官网怎么获得积分	界限突破贾诩	zaf

20000
30000
40000
50000
60000
70000
80000
90000
100000


In [28]:
df_stack = pd.DataFrame(index=range(len(df_all)))

# -----------------------stack for education/age/gender------------------
for lb in ['Education','age','gender']:
    print(lb)
    TR = 80000
    num_class = len(pd.value_counts(ys[lb]))
    n = 5

    X = X_sp[:TR]
    y = ys[lb][:TR]
    X_te = X_sp[TR:]
    y_te = ys[lb][TR:]

    stack = np.zeros((X.shape[0],num_class))
    stack_te = np.zeros((X_te.shape[0],num_class))

    for i,(tr,va) in enumerate(KFold(len(y),n_folds=n)):
        print('%s stack:%d/%d'%(str(datetime.now()),i+1,n))
        clf = LogisticRegression(C=3)
        clf.fit(X[tr],y[tr])
        y_pred_va = clf.predict_proba(X[va])
        y_pred_te = clf.predict_proba(X_te)
        print('va acc:',myAcc(y[va],y_pred_va))
        print('te acc:',myAcc(y_te,y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack,stack_te])
    for i in range(stack_all.shape[1]):
        df_stack['tfidf_{}_{}'.format(lb,i)] = stack_all[:,i]

df_stack.to_csv(cfg.data_path + 'tfidf_stack_10W.csv',index=None,encoding='utf8')
print(datetime.now(),'save tfidf stack done!')

Education
2018-09-18 22:16:03.477516 stack:1/5
va acc: 0.66125
te acc: 0.6602
2018-09-18 22:17:47.015585 stack:2/5
va acc: 0.6554375
te acc: 0.65835
2018-09-18 22:19:25.816432 stack:3/5
va acc: 0.65325
te acc: 0.65775
2018-09-18 22:21:02.842497 stack:4/5
va acc: 0.65475
te acc: 0.65995
2018-09-18 22:22:35.819534 stack:5/5
va acc: 0.656125
te acc: 0.66155
age
2018-09-18 22:24:08.836014 stack:1/5
va acc: 0.6038125
te acc: 0.6045
2018-09-18 22:25:52.097786 stack:2/5
va acc: 0.5934375
te acc: 0.60335
2018-09-18 22:27:37.722266 stack:3/5
va acc: 0.6003125
te acc: 0.6023
2018-09-18 22:29:23.797412 stack:4/5
va acc: 0.599375
te acc: 0.6025
2018-09-18 22:31:12.819629 stack:5/5
va acc: 0.60325
te acc: 0.60355
gender
2018-09-18 22:33:05.620973 stack:1/5
va acc: 0.8335
te acc: 0.8304
2018-09-18 22:33:22.806322 stack:2/5
va acc: 0.8331875
te acc: 0.83385
2018-09-18 22:33:39.735261 stack:3/5
va acc: 0.83075
te acc: 0.8332
2018-09-18 22:33:56.522526 stack:4/5
va acc: 0.833625
te acc: 0.83275
2018-09

In [29]:
'''dbow-nn stack for education/age/gender'''

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import KFold
from gensim.models import Doc2Vec
from collections import OrderedDict

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import re
import cfg

# ----------------------- myfunc -----------------------
def myAcc(y_true,y_pred):
    y_pred = np.argmax(y_pred,axis=1)
    return np.mean(y_true == y_pred)

# ----------------------- load dataset ----------------------
df_all = pd.read_csv(cfg.data_path + 'all_v2.csv',encoding='utf8',usecols=['Id','Education','age','gender'],nrows=100000)
ys = {}
for label in ['Education','age','gender']:
    ys[label] = np.array(df_all[label])
    
model = Doc2Vec.load(cfg.data_path + 'dbow_d2v.model')
X_sp = np.array([model.docvecs[i] for i in range(100000)])

C:\Users\sun\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [30]:
# ----------------------dbowd2v stack for Education/age/gender---------------------------
df_stack = pd.DataFrame(index=range(len(df_all)))
TR = 80000
n = 5

X = X_sp[:TR]
X_te = X_sp[TR:]

feat = 'dbowd2v'
for i,lb in enumerate(['Education','age','gender']):
    num_class = len(pd.value_counts(ys[lb]))
    y = ys[lb][:TR]
    y_te = ys[lb][TR:]
    
    stack = np.zeros((X.shape[0],num_class))
    stack_te = np.zeros((X_te.shape[0],num_class))
    
    for k,(tr,va) in enumerate(KFold(len(y),n_folds=n)):
        print('{} stack:{}/{}'.format(datetime.now(),k+1,n))
        nb_classes = num_class
        X_train = X[tr]
        y_train = y[tr]
        X_test = X_te
        y_test = y_te

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        Y_train = np_utils.to_categorical(y_train, nb_classes)
        Y_test = np_utils.to_categorical(y_test, nb_classes)


        model = Sequential()
        model.add(Dense(300,input_shape=(X_train.shape[1],)))
        model.add(Dropout(0.1))
        model.add(Activation('tanh'))
        model.add(Dense(nb_classes))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adadelta',
                      metrics=['accuracy'])

        history = model.fit(X_train, Y_train,shuffle=True,
                            batch_size=128, nb_epoch=35,
                            verbose=2, validation_data=(X_test, Y_test))
        y_pred_va = model.predict_proba(X[va])
        y_pred_te = model.predict_proba(X_te)
        print('va acc:',myAcc(y[va],y_pred_va))
        print('te acc:',myAcc(y_te,y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack,stack_te])
    for l in range(stack_all.shape[1]):
        df_stack['{}_{}_{}'.format(feat,lb,l)] = stack_all[:,l]
df_stack.to_csv(cfg.data_path + 'dbowd2v_stack_10W.csv',encoding='utf8',index=None)
print(datetime.now(),'save dbowd2v stack done!')

2018-09-18 22:44:08.239094 stack:1/5


C:\Users\sun\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 64000 samples, validate on 20000 samples
Epoch 1/35
 - 2s - loss: 0.9744 - acc: 0.6160 - val_loss: 0.9322 - val_acc: 0.6353
Epoch 2/35
 - 1s - loss: 0.9409 - acc: 0.6296 - val_loss: 0.9270 - val_acc: 0.6355
Epoch 3/35
 - 1s - loss: 0.9314 - acc: 0.6354 - val_loss: 0.9288 - val_acc: 0.6369
Epoch 4/35
 - 1s - loss: 0.9236 - acc: 0.6376 - val_loss: 0.9201 - val_acc: 0.6401
Epoch 5/35
 - 1s - loss: 0.9171 - acc: 0.6395 - val_loss: 0.9164 - val_acc: 0.6408
Epoch 6/35
 - 1s - loss: 0.9098 - acc: 0.6436 - val_loss: 0.9123 - val_acc: 0.6440
Epoch 7/35
 - 1s - loss: 0.9051 - acc: 0.6434 - val_loss: 0.9181 - val_acc: 0.6414
Epoch 8/35
 - 1s - loss: 0.8997 - acc: 0.6466 - val_loss: 0.9133 - val_acc: 0.6448
Epoch 9/35
 - 1s - loss: 0.8962 - acc: 0.6486 - val_loss: 0.9102 - val_acc: 0.6448
Epoch 10/35
 - 1s - loss: 0.8909 - acc: 0.6500 - val_loss: 0.9119 - val_acc: 0.6443
Epoch 11/35
 - 1s - loss: 0.8884 - acc: 0.6522 - val_loss: 0.9069 - val_acc: 0.6474
Epoch 12/35
 - 2s - loss: 0.8822 - 

Epoch 26/35
 - 1s - loss: 0.8222 - acc: 0.6786 - val_loss: 0.9182 - val_acc: 0.6461
Epoch 27/35
 - 1s - loss: 0.8171 - acc: 0.6803 - val_loss: 0.9264 - val_acc: 0.6473
Epoch 28/35
 - 1s - loss: 0.8118 - acc: 0.6831 - val_loss: 0.9274 - val_acc: 0.6441
Epoch 29/35
 - 1s - loss: 0.8053 - acc: 0.6845 - val_loss: 0.9262 - val_acc: 0.6451
Epoch 30/35
 - 1s - loss: 0.8025 - acc: 0.6876 - val_loss: 0.9285 - val_acc: 0.6466
Epoch 31/35
 - 1s - loss: 0.7949 - acc: 0.6889 - val_loss: 0.9362 - val_acc: 0.6397
Epoch 32/35
 - 1s - loss: 0.7879 - acc: 0.6922 - val_loss: 0.9377 - val_acc: 0.6429
Epoch 33/35
 - 1s - loss: 0.7844 - acc: 0.6935 - val_loss: 0.9416 - val_acc: 0.6437
Epoch 34/35
 - 1s - loss: 0.7784 - acc: 0.6958 - val_loss: 0.9474 - val_acc: 0.6380
Epoch 35/35
 - 1s - loss: 0.7704 - acc: 0.6987 - val_loss: 0.9438 - val_acc: 0.6393
va acc: 0.63875
te acc: 0.6393
2018-09-18 22:46:38.383836 stack:4/5
Train on 64000 samples, validate on 20000 samples
Epoch 1/35
 - 2s - loss: 0.9706 - acc: 0.6

Epoch 15/35
 - 1s - loss: 0.9721 - acc: 0.6202 - val_loss: 1.0021 - val_acc: 0.6135
Epoch 16/35
 - 1s - loss: 0.9664 - acc: 0.6239 - val_loss: 1.0048 - val_acc: 0.6135
Epoch 17/35
 - 1s - loss: 0.9625 - acc: 0.6254 - val_loss: 1.0032 - val_acc: 0.6135
Epoch 18/35
 - 1s - loss: 0.9591 - acc: 0.6265 - val_loss: 1.0223 - val_acc: 0.6113
Epoch 19/35
 - 2s - loss: 0.9542 - acc: 0.6297 - val_loss: 1.0084 - val_acc: 0.6122
Epoch 20/35
 - 1s - loss: 0.9497 - acc: 0.6304 - val_loss: 1.0095 - val_acc: 0.6095
Epoch 21/35
 - 1s - loss: 0.9457 - acc: 0.6328 - val_loss: 1.0110 - val_acc: 0.6108
Epoch 22/35
 - 1s - loss: 0.9370 - acc: 0.6362 - val_loss: 1.0337 - val_acc: 0.6085
Epoch 23/35
 - 1s - loss: 0.9336 - acc: 0.6370 - val_loss: 1.0108 - val_acc: 0.6111
Epoch 24/35
 - 2s - loss: 0.9275 - acc: 0.6406 - val_loss: 1.0170 - val_acc: 0.6107
Epoch 25/35
 - 2s - loss: 0.9238 - acc: 0.6429 - val_loss: 1.0144 - val_acc: 0.6092
Epoch 26/35
 - 1s - loss: 0.9175 - acc: 0.6446 - val_loss: 1.0194 - val_acc:

Epoch 4/35
 - 2s - loss: 1.0299 - acc: 0.5995 - val_loss: 1.0228 - val_acc: 0.6005
Epoch 5/35
 - 1s - loss: 1.0212 - acc: 0.6018 - val_loss: 1.0170 - val_acc: 0.6073
Epoch 6/35
 - 1s - loss: 1.0146 - acc: 0.6042 - val_loss: 1.0141 - val_acc: 0.6084
Epoch 7/35
 - 2s - loss: 1.0086 - acc: 0.6061 - val_loss: 1.0121 - val_acc: 0.6089
Epoch 8/35
 - 1s - loss: 1.0037 - acc: 0.6091 - val_loss: 1.0058 - val_acc: 0.6073
Epoch 9/35
 - 1s - loss: 0.9985 - acc: 0.6119 - val_loss: 1.0080 - val_acc: 0.6058
Epoch 10/35
 - 1s - loss: 0.9947 - acc: 0.6128 - val_loss: 1.0056 - val_acc: 0.6100
Epoch 11/35
 - 1s - loss: 0.9896 - acc: 0.6136 - val_loss: 1.0068 - val_acc: 0.6120
Epoch 12/35
 - 1s - loss: 0.9858 - acc: 0.6156 - val_loss: 1.0026 - val_acc: 0.6109
Epoch 13/35
 - 1s - loss: 0.9812 - acc: 0.6187 - val_loss: 1.0053 - val_acc: 0.6123
Epoch 14/35
 - 1s - loss: 0.9780 - acc: 0.6195 - val_loss: 1.0074 - val_acc: 0.6116
Epoch 15/35
 - 1s - loss: 0.9722 - acc: 0.6208 - val_loss: 1.0061 - val_acc: 0.610

Epoch 29/35
 - 1s - loss: 0.3750 - acc: 0.8512 - val_loss: 0.4135 - val_acc: 0.8414
Epoch 30/35
 - 1s - loss: 0.3724 - acc: 0.8511 - val_loss: 0.4154 - val_acc: 0.8377
Epoch 31/35
 - 1s - loss: 0.3707 - acc: 0.8515 - val_loss: 0.4185 - val_acc: 0.8397
Epoch 32/35
 - 1s - loss: 0.3681 - acc: 0.8532 - val_loss: 0.4175 - val_acc: 0.8393
Epoch 33/35
 - 1s - loss: 0.3661 - acc: 0.8544 - val_loss: 0.4229 - val_acc: 0.8374
Epoch 34/35
 - 1s - loss: 0.3646 - acc: 0.8539 - val_loss: 0.4211 - val_acc: 0.8401
Epoch 35/35
 - 1s - loss: 0.3608 - acc: 0.8558 - val_loss: 0.4336 - val_acc: 0.8347
va acc: 0.8308125
te acc: 0.83475
2018-09-18 22:53:32.291557 stack:2/5
Train on 64000 samples, validate on 20000 samples
Epoch 1/35
 - 2s - loss: 0.4388 - acc: 0.8272 - val_loss: 0.4300 - val_acc: 0.8355
Epoch 2/35
 - 1s - loss: 0.4256 - acc: 0.8357 - val_loss: 0.4241 - val_acc: 0.8367
Epoch 3/35
 - 1s - loss: 0.4207 - acc: 0.8368 - val_loss: 0.4214 - val_acc: 0.8398
Epoch 4/35
 - 1s - loss: 0.4170 - acc: 0.8

Epoch 18/35
 - 1s - loss: 0.3934 - acc: 0.8446 - val_loss: 0.4111 - val_acc: 0.8424
Epoch 19/35
 - 1s - loss: 0.3920 - acc: 0.8456 - val_loss: 0.4099 - val_acc: 0.8420
Epoch 20/35
 - 1s - loss: 0.3910 - acc: 0.8457 - val_loss: 0.4051 - val_acc: 0.8417
Epoch 21/35
 - 1s - loss: 0.3876 - acc: 0.8467 - val_loss: 0.4076 - val_acc: 0.8427
Epoch 22/35
 - 1s - loss: 0.3881 - acc: 0.8462 - val_loss: 0.4096 - val_acc: 0.8432
Epoch 23/35
 - 1s - loss: 0.3856 - acc: 0.8480 - val_loss: 0.4099 - val_acc: 0.8435
Epoch 24/35
 - 1s - loss: 0.3838 - acc: 0.8475 - val_loss: 0.4195 - val_acc: 0.8394
Epoch 25/35
 - 1s - loss: 0.3827 - acc: 0.8475 - val_loss: 0.4122 - val_acc: 0.8404
Epoch 26/35
 - 1s - loss: 0.3815 - acc: 0.8480 - val_loss: 0.4098 - val_acc: 0.8422
Epoch 27/35
 - 1s - loss: 0.3786 - acc: 0.8501 - val_loss: 0.4159 - val_acc: 0.8420
Epoch 28/35
 - 1s - loss: 0.3774 - acc: 0.8500 - val_loss: 0.4112 - val_acc: 0.8411
Epoch 29/35
 - 1s - loss: 0.3758 - acc: 0.8500 - val_loss: 0.4192 - val_acc:

In [32]:
'''dm-nn stack for education/age/gender'''

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cross_validation import KFold
from gensim.models import Doc2Vec
from collections import OrderedDict

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import re
import cfg

#-----------------------myfunc-----------------------
def myAcc(y_true,y_pred):
    y_pred = np.argmax(y_pred,axis=1)
    return np.mean(y_true == y_pred)

#-----------------------load dataset----------------------
df_all = pd.read_csv(cfg.data_path + 'all_v2.csv',encoding='utf8',usecols=['Id','Education','age','gender'],nrows=100000)
ys = {}
for label in ['Education','age','gender']:
    ys[label] = np.array(df_all[label])
    
model = Doc2Vec.load(cfg.data_path + 'dm_d2v.model')
X_sp = np.array([model.docvecs[i] for i in range(100000)])

In [33]:
#----------------------dmd2v stack for Education/age/gender---------------------------

df_stack = pd.DataFrame(index=range(len(df_all)))
TR = 80000
n = 5

X = X_sp[:TR]
X_te = X_sp[TR:]

feat = 'dmd2v'
for i,lb in enumerate(['Education','age','gender']):
    num_class = len(pd.value_counts(ys[lb]))
    y = ys[lb][:TR]
    y_te = ys[lb][TR:]
    
    stack = np.zeros((X.shape[0],num_class))
    stack_te = np.zeros((X_te.shape[0],num_class))
    
    for k,(tr,va) in enumerate(KFold(len(y),n_folds=n)):
        print('{} stack:{}/{}'.format(datetime.now(),k+1,n))
        nb_classes = num_class
        X_train = X[tr]
        y_train = y[tr]
        X_test = X_te
        y_test = y_te

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        Y_train = np_utils.to_categorical(y_train, nb_classes)
        Y_test = np_utils.to_categorical(y_test, nb_classes)


        model = Sequential()
        model.add(Dense(300,input_shape=(X_train.shape[1],)))
        model.add(Dropout(0.1))
        model.add(Activation('tanh'))
        model.add(Dense(nb_classes))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy',
                      optimizer='adadelta',
                      metrics=['accuracy'])

        history = model.fit(X_train, Y_train,shuffle=True,
                            batch_size=128, nb_epoch=35,
                            verbose=2, validation_data=(X_test, Y_test))
        y_pred_va = model.predict_proba(X[va])
        y_pred_te = model.predict_proba(X_te)
        print('va acc:',myAcc(y[va],y_pred_va))
        print('te acc:',myAcc(y_te,y_pred_te))
        stack[va] += y_pred_va
        stack_te += y_pred_te
    stack_te /= n
    stack_all = np.vstack([stack,stack_te])
    for l in range(stack_all.shape[1]):
        df_stack['{}_{}_{}'.format(feat,lb,l)] = stack_all[:,l]
df_stack.to_csv(cfg.data_path + 'dmd2v_stack_10W.csv',encoding='utf8',index=None)
print(datetime.now(),'save dmd2v stack done!')

2018-09-18 23:02:12.276668 stack:1/5


C:\Users\sun\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 64000 samples, validate on 20000 samples
Epoch 1/35
 - 2s - loss: 1.0232 - acc: 0.5899 - val_loss: 0.9509 - val_acc: 0.6243
Epoch 2/35
 - 2s - loss: 0.9356 - acc: 0.6279 - val_loss: 0.9436 - val_acc: 0.6282
Epoch 3/35
 - 2s - loss: 0.9008 - acc: 0.6434 - val_loss: 0.9532 - val_acc: 0.6281
Epoch 4/35
 - 2s - loss: 0.8658 - acc: 0.6585 - val_loss: 0.9523 - val_acc: 0.6290
Epoch 5/35
 - 2s - loss: 0.8279 - acc: 0.6763 - val_loss: 0.9735 - val_acc: 0.6232
Epoch 6/35
 - 2s - loss: 0.7829 - acc: 0.6952 - val_loss: 0.9761 - val_acc: 0.6280
Epoch 7/35
 - 2s - loss: 0.7410 - acc: 0.7141 - val_loss: 1.0004 - val_acc: 0.6250
Epoch 8/35
 - 2s - loss: 0.6979 - acc: 0.7341 - val_loss: 1.0287 - val_acc: 0.6180
Epoch 9/35
 - 2s - loss: 0.6567 - acc: 0.7521 - val_loss: 1.0644 - val_acc: 0.6076
Epoch 10/35
 - 2s - loss: 0.6216 - acc: 0.7629 - val_loss: 1.0977 - val_acc: 0.6100
Epoch 11/35
 - 2s - loss: 0.5856 - acc: 0.7785 - val_loss: 1.1169 - val_acc: 0.6066
Epoch 12/35
 - 2s - loss: 0.5527 - 

Epoch 26/35
 - 2s - loss: 0.3167 - acc: 0.8799 - val_loss: 1.5832 - val_acc: 0.5874
Epoch 27/35
 - 2s - loss: 0.3051 - acc: 0.8853 - val_loss: 1.6142 - val_acc: 0.5824
Epoch 28/35
 - 2s - loss: 0.2983 - acc: 0.8873 - val_loss: 1.6622 - val_acc: 0.5861
Epoch 29/35
 - 2s - loss: 0.2888 - acc: 0.8910 - val_loss: 1.6841 - val_acc: 0.5707
Epoch 30/35
 - 2s - loss: 0.2823 - acc: 0.8939 - val_loss: 1.6784 - val_acc: 0.5869
Epoch 31/35
 - 2s - loss: 0.2767 - acc: 0.8961 - val_loss: 1.7360 - val_acc: 0.5812
Epoch 32/35
 - 2s - loss: 0.2681 - acc: 0.9000 - val_loss: 1.7248 - val_acc: 0.5839
Epoch 33/35
 - 2s - loss: 0.2620 - acc: 0.9027 - val_loss: 1.7810 - val_acc: 0.5859
Epoch 34/35
 - 2s - loss: 0.2566 - acc: 0.9041 - val_loss: 1.7654 - val_acc: 0.5844
Epoch 35/35
 - 2s - loss: 0.2506 - acc: 0.9060 - val_loss: 1.7926 - val_acc: 0.5798
va acc: 0.55875
te acc: 0.5798
2018-09-18 23:05:08.484074 stack:4/5
Train on 64000 samples, validate on 20000 samples
Epoch 1/35
 - 2s - loss: 1.0276 - acc: 0.5

Epoch 15/35
 - 2s - loss: 0.5659 - acc: 0.7930 - val_loss: 1.3567 - val_acc: 0.5517
Epoch 16/35
 - 2s - loss: 0.5426 - acc: 0.7986 - val_loss: 1.3863 - val_acc: 0.5542
Epoch 17/35
 - 2s - loss: 0.5238 - acc: 0.8083 - val_loss: 1.4269 - val_acc: 0.5466
Epoch 18/35
 - 2s - loss: 0.5036 - acc: 0.8133 - val_loss: 1.4633 - val_acc: 0.5464
Epoch 19/35
 - 2s - loss: 0.4863 - acc: 0.8201 - val_loss: 1.4748 - val_acc: 0.5507
Epoch 20/35
 - 2s - loss: 0.4677 - acc: 0.8266 - val_loss: 1.5178 - val_acc: 0.5467
Epoch 21/35
 - 2s - loss: 0.4538 - acc: 0.8327 - val_loss: 1.5445 - val_acc: 0.5502
Epoch 22/35
 - 2s - loss: 0.4424 - acc: 0.8371 - val_loss: 1.5675 - val_acc: 0.5432
Epoch 23/35
 - 2s - loss: 0.4297 - acc: 0.8422 - val_loss: 1.6059 - val_acc: 0.5488
Epoch 24/35
 - 2s - loss: 0.4148 - acc: 0.8467 - val_loss: 1.6412 - val_acc: 0.5443
Epoch 25/35
 - 2s - loss: 0.4044 - acc: 0.8501 - val_loss: 1.6694 - val_acc: 0.5472
Epoch 26/35
 - 2s - loss: 0.3966 - acc: 0.8538 - val_loss: 1.6873 - val_acc:

Epoch 4/35
 - 2s - loss: 0.9689 - acc: 0.6261 - val_loss: 1.0590 - val_acc: 0.5914
Epoch 5/35
 - 2s - loss: 0.9241 - acc: 0.6454 - val_loss: 1.0661 - val_acc: 0.5881
Epoch 6/35
 - 2s - loss: 0.8805 - acc: 0.6655 - val_loss: 1.1093 - val_acc: 0.5727
Epoch 7/35
 - 2s - loss: 0.8336 - acc: 0.6854 - val_loss: 1.1302 - val_acc: 0.5790
Epoch 8/35
 - 2s - loss: 0.7873 - acc: 0.7045 - val_loss: 1.1437 - val_acc: 0.5754
Epoch 9/35
 - 2s - loss: 0.7469 - acc: 0.7204 - val_loss: 1.1655 - val_acc: 0.5696
Epoch 10/35
 - 2s - loss: 0.7079 - acc: 0.7379 - val_loss: 1.2113 - val_acc: 0.5693
Epoch 11/35
 - 2s - loss: 0.6727 - acc: 0.7513 - val_loss: 1.2599 - val_acc: 0.5685
Epoch 12/35
 - 2s - loss: 0.6388 - acc: 0.7638 - val_loss: 1.2856 - val_acc: 0.5646
Epoch 13/35
 - 2s - loss: 0.6140 - acc: 0.7725 - val_loss: 1.3714 - val_acc: 0.5525
Epoch 14/35
 - 2s - loss: 0.5847 - acc: 0.7847 - val_loss: 1.3528 - val_acc: 0.5613
Epoch 15/35
 - 2s - loss: 0.5625 - acc: 0.7934 - val_loss: 1.3817 - val_acc: 0.557

Epoch 29/35
 - 2s - loss: 0.0575 - acc: 0.9774 - val_loss: 1.0452 - val_acc: 0.7959
Epoch 30/35
 - 2s - loss: 0.0535 - acc: 0.9796 - val_loss: 1.0599 - val_acc: 0.7996
Epoch 31/35
 - 2s - loss: 0.0555 - acc: 0.9792 - val_loss: 1.0924 - val_acc: 0.7954
Epoch 32/35
 - 2s - loss: 0.0504 - acc: 0.9809 - val_loss: 1.0802 - val_acc: 0.7983
Epoch 33/35
 - 2s - loss: 0.0474 - acc: 0.9819 - val_loss: 1.0985 - val_acc: 0.7961
Epoch 34/35
 - 2s - loss: 0.0468 - acc: 0.9830 - val_loss: 1.1337 - val_acc: 0.8006
Epoch 35/35
 - 2s - loss: 0.0449 - acc: 0.9835 - val_loss: 1.1330 - val_acc: 0.8007
va acc: 0.78925
te acc: 0.80075
2018-09-18 23:12:59.311035 stack:2/5
Train on 64000 samples, validate on 20000 samples
Epoch 1/35
 - 2s - loss: 0.4699 - acc: 0.8027 - val_loss: 0.4497 - val_acc: 0.8175
Epoch 2/35
 - 2s - loss: 0.4290 - acc: 0.8240 - val_loss: 0.4407 - val_acc: 0.8218
Epoch 3/35
 - 2s - loss: 0.4130 - acc: 0.8315 - val_loss: 0.4415 - val_acc: 0.8203
Epoch 4/35
 - 2s - loss: 0.3940 - acc: 0.839

Epoch 18/35
 - 2s - loss: 0.1073 - acc: 0.9578 - val_loss: 0.8206 - val_acc: 0.7948
Epoch 19/35
 - 2s - loss: 0.0980 - acc: 0.9621 - val_loss: 0.8197 - val_acc: 0.8013
Epoch 20/35
 - 2s - loss: 0.0916 - acc: 0.9644 - val_loss: 0.8433 - val_acc: 0.8017
Epoch 21/35
 - 2s - loss: 0.0878 - acc: 0.9667 - val_loss: 0.8980 - val_acc: 0.8045
Epoch 22/35
 - 2s - loss: 0.0813 - acc: 0.9688 - val_loss: 0.8801 - val_acc: 0.7993
Epoch 23/35
 - 2s - loss: 0.0739 - acc: 0.9719 - val_loss: 0.9110 - val_acc: 0.7983
Epoch 24/35
 - 2s - loss: 0.0694 - acc: 0.9741 - val_loss: 1.0896 - val_acc: 0.7786
Epoch 25/35
 - 2s - loss: 0.0674 - acc: 0.9750 - val_loss: 0.9433 - val_acc: 0.8005
Epoch 26/35
 - 2s - loss: 0.0636 - acc: 0.9758 - val_loss: 0.9618 - val_acc: 0.8010
Epoch 27/35
 - 2s - loss: 0.0628 - acc: 0.9763 - val_loss: 0.9794 - val_acc: 0.7990
Epoch 28/35
 - 2s - loss: 0.0581 - acc: 0.9785 - val_loss: 1.0079 - val_acc: 0.8001
Epoch 29/35
 - 2s - loss: 0.0563 - acc: 0.9785 - val_loss: 0.9983 - val_acc:

In [35]:
'''xgb-ens for education/age/gender'''

import pandas as pd
import numpy as np
import xgboost as xgb
import cfg
import datetime

def xgb_acc_score(preds,dtrain):
    y_true = dtrain.get_label()
    y_pred = np.argmax(preds,axis=1)
    return [('acc',np.mean(y_true == y_pred))]

df_lr = pd.read_csv(cfg.data_path + 'tfidf_stack_10W.csv')
df_dm = pd.read_csv(cfg.data_path + 'dmd2v_stack_10W.csv')
df_dbow = pd.read_csv(cfg.data_path + 'dbowd2v_stack_10W.csv')

df_lb = pd.read_csv(cfg.data_path + 'all_v2.csv',usecols=['Id','Education','age','gender'],nrows=100000)
ys = {}
for lb in ['Education','age','gender']:
    ys[lb] = np.array(df_lb[lb])

'''最好的参数组合'''
#-------------------------education----------------------------------
TR = 80000
df_sub = pd.DataFrame()
df_sub['Id'] = df_lb.iloc[TR:]['Id']
seed = 10
lb = 'Education'
print(lb)

esr = 25
evals = 1
n_trees = 1000

df = pd.concat([df_lr,df_dbow,df_dm],axis=1)
print(df.columns)
num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]

ss = 0.9
mc = 2
md = 8
gm = 2
# n_trees = 30

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':0.8,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
#     'seed':seed,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['Education'] = np.argmax(bst.predict(dvalid),axis=1) + 1
#------------------------ age-----------------------------------
lb = 'age'
print(lb)
num_class = len(pd.value_counts(ys[lb]))

# df = pd.concat([df_stack_tfidf,df_stack_d2v],axis=1)
num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]

ss = 0.5
mc = 3
md = 7
gm = 2
# n_trees = 37

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':1,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
#     'seed':seed,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['age'] = np.argmax(bst.predict(dvalid),axis=1)+1
#--------------------------gender-------------------------------------
lb = 'gender'
print(lb)
num_class = len(pd.value_counts(ys[lb]))

# df = pd.concat([df_lr,df_multid2v],axis=1)
num_class = len(pd.value_counts(ys[lb]))
X = df.iloc[:TR]
y = ys[lb][:TR]
X_te = df.iloc[TR:]
y_te = ys[lb][TR:]


ss = 0.5
mc = 0.8
md = 7
gm = 1
# n_trees = 25

params = {
    "objective": "multi:softprob",
    "booster": "gbtree",
#     "eval_metric": "merror",
    "num_class":num_class,
    'max_depth':md,
    'min_child_weight':mc,
    'subsample':ss,
    'colsample_bytree':1,
    'gamma':gm,
    "eta": 0.01,
    "lambda":0,
    'alpha':0,
    "silent": 1,
#     'seed':seed,
}

dtrain = xgb.DMatrix(X, y)
dvalid = xgb.DMatrix(X_te, y_te)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
bst = xgb.train(params, dtrain, n_trees, evals=watchlist,feval=xgb_acc_score,maximize=True,
                early_stopping_rounds=esr, verbose_eval=evals)
df_sub['gender'] = np.argmax(bst.predict(dvalid),axis=1)+1

df_sub = df_sub[['Id','age','gender','Education']]
df_sub.to_csv(cfg.data_path + 'tfidf_dm_dbow_2W.csv',index=None,header=None,sep=' ')

Education
Index(['tfidf_Education_0', 'tfidf_Education_1', 'tfidf_Education_2',
       'tfidf_Education_3', 'tfidf_Education_4', 'tfidf_Education_5',
       'tfidf_age_0', 'tfidf_age_1', 'tfidf_age_2', 'tfidf_age_3',
       'tfidf_age_4', 'tfidf_age_5', 'tfidf_gender_0', 'tfidf_gender_1',
       'dbowd2v_Education_0', 'dbowd2v_Education_1', 'dbowd2v_Education_2',
       'dbowd2v_Education_3', 'dbowd2v_Education_4', 'dbowd2v_Education_5',
       'dbowd2v_age_0', 'dbowd2v_age_1', 'dbowd2v_age_2', 'dbowd2v_age_3',
       'dbowd2v_age_4', 'dbowd2v_age_5', 'dbowd2v_gender_0',
       'dbowd2v_gender_1', 'dmd2v_Education_0', 'dmd2v_Education_1',
       'dmd2v_Education_2', 'dmd2v_Education_3', 'dmd2v_Education_4',
       'dmd2v_Education_5', 'dmd2v_age_0', 'dmd2v_age_1', 'dmd2v_age_2',
       'dmd2v_age_3', 'dmd2v_age_4', 'dmd2v_age_5', 'dmd2v_gender_0',
       'dmd2v_gender_1'],
      dtype='object')
[0]	train-merror:0.312775	eval-merror:0.3326	train-acc:0.687225	eval-acc:0.6674
Multiple eva

[16]	train-merror:0.357825	eval-merror:0.3626	train-acc:0.642175	eval-acc:0.6374
[17]	train-merror:0.357675	eval-merror:0.3627	train-acc:0.642325	eval-acc:0.6373
[18]	train-merror:0.357513	eval-merror:0.36325	train-acc:0.642487	eval-acc:0.63675
[19]	train-merror:0.357475	eval-merror:0.36335	train-acc:0.642525	eval-acc:0.63665
[20]	train-merror:0.35745	eval-merror:0.36355	train-acc:0.64255	eval-acc:0.63645
[21]	train-merror:0.357388	eval-merror:0.36335	train-acc:0.642613	eval-acc:0.63665
[22]	train-merror:0.356813	eval-merror:0.3639	train-acc:0.643188	eval-acc:0.6361
[23]	train-merror:0.356875	eval-merror:0.3636	train-acc:0.643125	eval-acc:0.6364
[24]	train-merror:0.356813	eval-merror:0.36355	train-acc:0.643188	eval-acc:0.63645
[25]	train-merror:0.356775	eval-merror:0.36345	train-acc:0.643225	eval-acc:0.63655
[26]	train-merror:0.356638	eval-merror:0.3636	train-acc:0.643362	eval-acc:0.6364
[27]	train-merror:0.356463	eval-merror:0.36365	train-acc:0.643537	eval-acc:0.63635
[28]	train-merro

[72]	train-merror:0.14675	eval-merror:0.15145	train-acc:0.85325	eval-acc:0.84855
[73]	train-merror:0.14665	eval-merror:0.15155	train-acc:0.85335	eval-acc:0.84845
[74]	train-merror:0.14665	eval-merror:0.1515	train-acc:0.85335	eval-acc:0.8485
[75]	train-merror:0.146675	eval-merror:0.1514	train-acc:0.853325	eval-acc:0.8486
[76]	train-merror:0.14665	eval-merror:0.1514	train-acc:0.85335	eval-acc:0.8486
[77]	train-merror:0.146725	eval-merror:0.15155	train-acc:0.853275	eval-acc:0.84845
[78]	train-merror:0.146575	eval-merror:0.15165	train-acc:0.853425	eval-acc:0.84835
[79]	train-merror:0.146625	eval-merror:0.1515	train-acc:0.853375	eval-acc:0.8485
[80]	train-merror:0.146637	eval-merror:0.1514	train-acc:0.853363	eval-acc:0.8486
[81]	train-merror:0.146637	eval-merror:0.15145	train-acc:0.853363	eval-acc:0.84855
[82]	train-merror:0.146575	eval-merror:0.15165	train-acc:0.853425	eval-acc:0.84835
[83]	train-merror:0.146563	eval-merror:0.15165	train-acc:0.853437	eval-acc:0.84835
[84]	train-merror:0.14